In [1]:
import sys
import os
sys.path.append("../")
import numpy as np
import pandas as pd
from sklearn.model_selection import RepeatedKFold

from sklearn.metrics import roc_auc_score, accuracy_score
import optuna
import optuna.integration.lightgbm as lgb

from data_loader import lgbm_data_loader, category_data_loader



/opt/conda/envs/lgbm/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
IS_CUSTOM = True
USE_VALID = True
USE_PCA = False
valid_len = 3
n_components = 5

DROPS = [
    'Timestamp','year','day','minute','second','KnowledgeTag',
]
PCA = [
    # 'hour_answerCode_sum',
    # 'userID_dayofweek_answerCode_count',
    # 'user_correct_answer',
    # 'user_total_answer',
    # 'hour_answerCode_var',
    # 'hour_answerCode_mean',
    # 'userID_first3_answerCode_count',
    # 'userID_month_answerCode_count',
    # 'KnowledgeTag_first3_answerCode_sum',
    # 'KnowledgeTag',
    # 'userID_answerCode_count',
    # 'userID_answerCode_sum',
    # 'testId_answerCode_sum',
    # 'KnowledgeTag_answerCode_count',
    # 'KnowledgeTag_answerCode_sum',
    # 'month',
    # 'hour',
    # 'dayofweek',
    # 'dayofweek_answerCode_mean',
    # 'dayofweek_answerCode_count',
    # 'dayofweek_answerCode_sum',
    # 'dayofweek_answerCode_var',
    # 'mid3',
    # 'KnowledgeTag_first3_answerCode_mean',
    # 'KnowledgeTag_first3_answerCode_count',
    # 'month_answerCode_var',
    # 'month_answerCode_count',
    # 'hour_answerCode_count', 
    # 'KnowledgeTag_first3_answerCode_var',
]

In [3]:
# train, valid, y_train, y_valid, test = lgbm_data_loader(IS_CUSTOM=IS_CUSTOM,USE_VALID=USE_VALID, DROPS=DROPS, valid_len=valid_len)
train, valid, y_train, y_valid, test, ids_of_categorical = category_data_loader(IS_CUSTOM=IS_CUSTOM, USE_VALID=USE_VALID, DROPS=DROPS, valid_len=valid_len,PCA=PCA, n_components=5, USE_PCA=USE_PCA)
# train, valid, y_train, y_valid, test = ctb_data_loader(IS_CUSTOM=IS_CUSTOM,USE_VALID=USE_VALID, DROPS=DROPS+low_importance, low_importance=low_importance, n_components=5, valid_len=3)

Label Encoding...: 100%|██████████| 69/69 [00:16<00:00,  4.12it/s]


Start binning


Binning...: 100%|██████████| 20/20 [00:32<00:00,  1.64s/it]


In [4]:
rkf = RepeatedKFold(n_splits=10, n_repeats=10, random_state=42)

params = {
        "objective": "binary",
        "metric": "binary_error",
        "verbosity": -1,
        "boosting_type": "gbdt",                
        "seed": 42
    }

X = train    
y = y_train

study_tuner = optuna.create_study(direction='minimize')
dtrain = lgb.Dataset(X, label=y)

# Suppress information only outputs - otherwise optuna is 
# quite verbose, which can be nice, but takes up a lot of space
optuna.logging.set_verbosity(optuna.logging.WARNING) 

tuner = lgb.LightGBMTunerCV(params, 
                            dtrain, 
                            categorical_feature=ids_of_categorical,
                            study=study_tuner,
                            verbose_eval=False,                            
                            early_stopping_rounds=250,
                            time_budget=19800, # Time budget of 5 hours, we will not really need it
                            seed = 42,
                            folds=rkf,
                            num_boost_round=10000,
                            callbacks=[lgb.reset_parameter(learning_rate = [0.005]*200 + [0.001]*9800) ] #[0.1]*5 + [0.05]*15 + [0.01]*45 + 
                           )

tuner.run()

[I 2022-11-30 13:49:12,971] A new study created in memory with name: no-name-2344c2dd-e8ac-4a5f-af8a-147028611475
feature_fraction, val_score: inf:   0%|          | 0/7 [00:00<?, ?it/s]/opt/conda/envs/lgbm/lib/python3.10/site-packages/lightgbm/engine.py:577: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/envs/lgbm/lib/python3.10/site-packages/lightgbm/basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['KnowledgeTag_answerCode_count', 'KnowledgeTag_answerCode_mean', 'KnowledgeTag_answerCode_sum', 'KnowledgeTag_answerCode_var', 'KnowledgeTag_first3_answerCode_count', 'KnowledgeTag_first3_answerCode_mean', 'KnowledgeTag_first3_answerCode_sum', 'KnowledgeTag_first3_answerCode_var', 

KeyboardInterrupt: 

In [ ]:
print(tuner.best_params)
# Classification error
print(tuner.best_score)
# Or expressed as accuracy
print(1.0-tuner.best_score)

In [ ]:
tmp_best_params = tuner.best_params

if tmp_best_params['feature_fraction']==1:
    tmp_best_params['feature_fraction']=1.0-1e-9
if tmp_best_params['feature_fraction']==0:
    tmp_best_params['feature_fraction']=1e-9
if tmp_best_params['bagging_fraction']==1:
    tmp_best_params['bagging_fraction']=1.0-1e-9
if tmp_best_params['bagging_fraction']==0:
    tmp_best_params['bagging_fraction']=1e-9  

In [ ]:
import lightgbm as lgb
dtrain = lgb.Dataset(X, label=y)

# We will track how many training rounds we needed for our best score.
# We will use that number of rounds later.
best_score = 999
training_rounds = 10000

# Declare how we evaluate how good a set of hyperparameters are, i.e.
# declare an objective function.
def objective(trial):
    # Specify a search space using distributions across plausible values of hyperparameters.
    param = {
        "objective": "binary",
        "metric": "binary_error",
        "verbosity": -1,
        "boosting_type": "gbdt",                
        "seed": 42,
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
        'num_leaves': trial.suggest_int('num_leaves', 2, 512),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.1, 1.0),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.1, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 0, 15),
        'min_child_samples': trial.suggest_int('min_child_samples', 1, 100),
        'seed': 1979
    }
    lgbcv = lgb.cv(param,
                   dtrain,
                   categorical_feature=ids_of_categorical,
                   folds=rkf,
                   verbose_eval=False,                   
                   early_stopping_rounds=250,                   
                   num_boost_round=10000,                    
                   callbacks=[lgb.reset_parameter(learning_rate = [0.005]*200 + [0.001]*9800) ]
                  )
    
    cv_score = lgbcv['binary_error-mean'][-1] + lgbcv['binary_error-stdv'][-1]
    if cv_score<best_score:
        training_rounds = len( list(lgbcv.values())[0] )
    
    # Return metric of interest
    return cv_score
optuna.logging.set_verbosity(optuna.logging.WARNING) 

# We search for another 4 hours (3600 s are an hours, so timeout=14400).
# We could instead do e.g. n_trials=1000, to try 1000 hyperparameters chosen 
# by optuna or set neither timeout or n_trials so that we keep going until 
# the user interrupts ("Cancel run").
study = optuna.create_study(direction='maximize')  
study.enqueue_trial(tmp_best_params)
study.optimize(objective, timeout=14400) 